# Contract risk analysis

In [5]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");

record Car (string make, string model, string basemodel, string color, int year, string condition, int mileage, int price);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [6]:
async Task<string> process_competion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Render an Scriban template

In [7]:
string render_tempate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

In [12]:
//# Create a semantic kernel inline function
var faq_template = @"System:
You are an agent that can help generate FAQ documents.
Today's date is {{date}}.

User:
For the following text, generate a detailed FAQ document titled 'Contoso Company Benefits FAQ'. 

Text: ===
{{input}}
===

Sample Output format: ===
Title: // title
Date: // Current date

Q: What is 401k?
A:  A: A 401k plan is a retirement savings plan that allows you to contribute a portion of your pre-tax income to a designated account.
===

Use only the provided text. Do not use any external sources.";

var input = "test";
var datestr = DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss");
var prompt = render_tempate(faq_template, new {input=input, date=datestr});
Console.WriteLine(prompt);

System:
You are an agent that can help generate FAQ documents.
Today's date is 2024-03-07 11:35:28.

User:
For the following text, generate a detailed FAQ document titled 'Contoso Company Benefits FAQ'. 

Text: ===
test
===

Sample Output format: ===
Title: // title
Date: // Current date

Q: What is 401k?
A:  A: A 401k plan is a retirement savings plan that allows you to contribute a portion of your pre-tax income to a designated account.
===

Use only the provided text. Do not use any external sources.


### Proces Prompt and Completion

In [11]:
// Create a semantic kernel inline function
var sales_desc_generation_template = "Create a one paragraph sales description that includes the price for a {{input}}";
var prompt = render_tempate(sales_desc_generation_template, new {input = description});


// Execute the SK function
Console.WriteLine(await process_competion(prompt, 200));

Introducing the stunning 2019 Ford Explorer XLT in a vibrant red color, now available for just $25,000! With only 32,000 miles on the odometer, this well-maintained vehicle is in excellent condition and ready to take you on your next adventure. The Explorer XLT offers a spacious and comfortable interior, advanced safety features, and powerful performance, making it the perfect choice for families or anyone in need of a reliable and stylish SUV. Don't miss out on this incredible deal - come test drive the 2019 Ford Explorer XLT today!
